In [1]:
from tweepy import OAuthHandler

from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd 
# import csv
# import re 
# from textblob import TextBlob
# import string 
# import preprocessor as p 
import os 
import time


In [2]:
with open('./twitter_credentials.json') as jsonkeys:
    data= json.load(jsonkeys)



In [3]:
consumer_key = data['CONSUMER_KEY']
consumer_secret = data['CONSUMER_SECRET']
access_key = data['ACCESS_TOKEN']
access_secret = data['ACCESS_SECRET']


#passing your twitter credentials to tweepy via Oauthandler

auth = OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_key,access_secret)
api = tweepy.API(auth)

In [4]:
def scraptweets(search_words, date_since,numTweets,numRuns):
    #using a for loop to generate tweets at regular intervals 
    #defining a pandas dataframe to store date
    
    
    db_tweets  = pd.DataFrame(columns =['username','acctdesc','location','following','followers',
                                        'totaltweets','usercreatedts','tweetcreatedts','retweetcount',
                                       'text','hashtags'])
    
    program_start = time.time()
    for i in range(0,numRuns):
        #we will time how long it takes to tweets tweets for each run
        start_run = time.time()
        
        #collect tweets using the Cursor Object
        
        tweets = tweepy.Cursor(api.search, q = search_words, lang='en',since = date_since,
                              tweet_mode = 'extended').items(numTweets)
        
        #store thsee tweets into a python list 
        tweet_list = [tweet for tweet in tweets]
        
        #obtain the following info
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
        
        
        
        
        #begin scrapping the tweets individually:
        numTweets = 0 
        for tweet in tweet_list:
            #pull the values
            username= tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts= tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            
            # everything from the begining of try block to where time.sleep() is under this for tweet in tweets loop
            try:

                text = tweet.retweeted_status.full_text

            except AttributeError:
                text = tweet.full_text

                #add the  11 variables to the empty list - ith_tweet
            ith_tweet = [username, acctdesc, location,following,followers,
                        totaltweets,usercreatedts,tweetcreatedts,retweetcount,
                        text, hashtags]

            #append to dataframe
            db_tweets.loc[len(db_tweets)] = ith_tweet

            #increase counter - noTweets
            numTweets += 1

            #run ended


            end_run = time.time()
            duration_run = round((end_run - start_run)/60,2)
            print (f'time taken for {i+1} run to complete is {duration_run}')

#             time.sleep(920)
        
        from datetime import datetime
        
        #obtain timestamp in readable format
        
        to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
        
        #define working path and filename
        path = os.getcwd()
        filename = path + '/data' + to_csv_timestamp + '70daysbootcamp_tweets.csv'
        
        #store dataframe in csv with creation date timestamp
        
        db_tweets.to_csv(filename, index = False)
        
    # and everything from here to the end is not under any loop
    program_end = time.time()
    print('Scrapping has completed!')
    print(f'Total time taken to scrap is {round((program_end - program_start)/60, 2)} minutes.')        

In [5]:
#Intialising these variables

search_words = '#70daysofML OR #AIbootcamp2020 OR #DSN70daysofML OR #70daysofDL OR #DSN70daysofDL OR #1_million_AI_talents_in_10_year OR #4-day_scorestreak'



date_since = '2020-07-27'

numTweets = 200

numRuns= 1

In [1]:
scraptweets(search_words,date_since,numTweets,numRuns)

In [8]:
df= pd.read_csv('data20200804_15140170daysbootcamp_tweets.csv')
df.head() 

,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags
0,Dsn70daysofML,I am a bot that was created to inspire the nex...,"Lagos, Nigeria",22,66,506,2020-06-29 07:01:47,2020-08-04 13:51:56,1,#Day35 of #DSN70daysofML\n\nProgress report of...,"[{'text': 'Day35', 'indices': [15, 21]}, {'tex..."
1,fikkyman1,Christ! 📢\nTech boy 💻\nUnited ✨✨,Nigeria,417,74,293,2014-09-06 18:56:18,2020-08-04 13:51:42,1,#Day35 of #DSN70daysofML\n\nProgress report of...,"[{'text': 'Day35', 'indices': [0, 6]}, {'text'..."
2,Pardoe_AI,"#AI #Influencer, #Advisor & #Investor. Host ht...",Global,7406,15150,185750,2014-07-06 12:41:41,2020-08-04 04:06:06,7,Day 34 #DSN70DaysOfML. And we've moved to K me...,"[{'text': 'DSN70DaysOfML', 'indices': [21, 35]}]"
3,youngapp_pf,AI-driven text data extracting APIs for busine...,"Paris, France",3608,2022,31400,2015-11-29 14:15:48,2020-08-04 00:48:53,7,Day 34 #DSN70DaysOfML. And we've moved to K me...,"[{'text': 'DSN70DaysOfML', 'indices': [21, 35]}]"
4,o_funminiyi,Embedded Systems | Internet of Things | Artifi...,NaN,739,559,1684,2019-05-14 10:29:12,2020-08-04 00:12:40,4,Day[34/70]\nUnsupervised Learning...Clustering...,[]
